In [1]:
#Imports
import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown,display,update_display
from openai import OpenAI

In [5]:
#Load Enviroment variables in a file called .env and Intialize Contstans
load_dotenv()
api_key = os.getenv('OPEN_API_KEY')
#Check the key
if not api_key:
    print("No Api key was found")
elif api_key.strip() !=api_key:
    print("An API key was foun,but it looks like it might have a space or tab charachters at the start or end-please remove it")
else:
    print("API key found and looks good so far!")
MODEL='gpt-4o-mini'
openai = OpenAI(api_key=api_key)

API key found and looks good so far!


In [6]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"
    

In [11]:
cnn=Website("https://cnn.com")
print(ed.get_contents())

Webpage Title:
Home - Edward Donner
Webpage Contents:
Home
Connect Four
Outsmart
An arena that pits LLMs against each other in a battle of diplomacy and deviousness
About
Posts
Well, hi there.
I’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (
very
amateur) and losing myself in
Hacker News
, nodding my head sagely to things I only half understand.
I’m the co-founder and CTO of
Nebula.io
. We’re applying AI to a field where it can make a massive, positive impact: helping people discover their potential and pursue their reason for being. Recruiters use our product today to source, understand, engage and manage talent. I’m previously the founder and CEO of AI startup untapt,
acquired in 2021
.
We work with groundbreaking, proprietary LLMs verticalized for talent, we’ve
patented
our matching model, and our award-winning platform has happy customers a

In [25]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""

In [26]:
print(link_system_prompt)

You are provided with a list of links found on a webpage. You are able to decide which of the links would be most relevant to include in a brochure about the company, such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}



In [27]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [28]:
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [29]:
get_links("https://www.tgju.org/currency")

{'links': [{'type': 'homepage', 'url': 'https://www.tgju.org'},
  {'type': 'careers page', 'url': 'https://careers.tgju.org/fa/career'},
  {'type': 'about page', 'url': 'https://www.tgju.org/applications'},
  {'type': 'contact page', 'url': 'https://www.tgju.org/contact'},
  {'type': 'glossary page', 'url': 'https://www.tgju.org/glossary/economics'}]}

In [30]:
#Make the brochure
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result



In [33]:
#print(get_all_details('https://cnn.com'))

In [57]:
#Second Prompt
#system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
#and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
#Include details of company culture, customers and careers/jobs if you have the information."
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website and find team and name of team \
and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."

In [58]:
#Function for get_brochure
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [59]:
#Function for Create a Brochure
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [54]:
create_brochure("Bogariagency", "http://bogariagency.com/")

Found links: {'links': [{'type': 'about page', 'url': 'http://bogariagency.com/about-farideh/'}, {'type': 'development page', 'url': 'http://bogariagency.com/development/'}, {'type': 'contact page', 'url': 'http://bogariagency.com/contact/'}, {'type': 'properties page', 'url': 'http://bogariagency.com/properties/'}, {'type': 'testimonials page', 'url': 'http://bogariagency.com/testimonials/'}, {'type': 'press page', 'url': 'http://bogariagency.com/press/'}]}


```markdown
# Welcome to Bogari International Realty! 🌟

## Your Dream Home Awaits! 🏡

Hey there, future homeowner, investor, or perhaps the world’s next real estate superstar! If you’re combing through the vast expanse of Los Angeles real estate like a toddler searching for toys in a messy room, you’ve stumbled upon the *magical entity* known as **Bogari International Realty**—where we don’t just buy and sell properties; we cook up dreams with a pinch of luxury and a dash of elegance!

---

### Who is Faye Bogari? 🍸

Meet *Farideh (Faye) Bogari*, the mastermind behind Bogari International Realty! With over two decades steering through the tricky waters of the L.A. property market, she’s like the Superwoman of real estate—complete with a cape made of high-end contracts! 🦸‍♀️

Faye doesn’t just acquire and develop properties; she transforms spaces that would make even *Bob the Builder* raise a glass! From residential palaces to commercial empires, she’s your gal.

---

### Why Choose Us? 🎉

- **Expertise Like No Other:** Our Faye has the *knowledge* of the market, the *skills* of a ninja negotiator, and a heart so big it could contain the Los Angeles skyline.
  
- **Client-Centric Approach:** As one happy tenant summed it up, “Faye makes everything easy.” And if you ask nicely, she might even throw in a rooftop mini-golf course! (Just kidding... or are we?)

- **Prime Locations:** Beverly Hills? Check! New York? You betcha! Vienna? Oui, oui! We’re basically your international ticket to living fabulously.

---

### Who's Our Typical Customer? 👥

Everyone from first-time homebuyers to seasoned investors, and the coolest consulates around — yes, we cater to *all* sorts! Our clients love us, and they’re not shy about showing it. Don’t believe us? Just check out the testimonials; we’re practically blushing over here.

**Testimonial Shoutout! 🎙️**
*“Faye is not just a realtor; she’s a magician! She turned my complicated embassy deal into a piece of cake, and I’m still waiting for the frosting!”* - Hassan Shawaf

---

### Join the Bogari Family! 👩‍👩‍👦

Thinking about joining our incredible team? Here’s what you'll encounter when you step through our not-so-secret doors (which may or may not be lined with gold):

- A company culture that’s as welcoming as a grandma’s hugs! We're serious about hard work and generous with the fun!
- Opportunities to work alongside real estate royalty and learn from the cleverly crafty Faye (watch out; she keeps chocolate on her desk as a motivator!).
- A vibrant, proactive atmosphere where both the coffee and the ideas flow freely. ☕💡

---

### In Conclusion: 

Whether you’re looking for a home sweeter than your grandma’s cookies 🍪, seeking an investment opportunity that’ll make your wallet sing 🎶, or searching for a workplace where every day is an “Insta-worthy” adventure 📸, **Bogari International Realty** has got your back!

Connect with us at:  
📞 **+1.310.717.0029**  
📧 **info(at)bogari(dotted)agency**

Come on, future homeowner-investor-superstar! Let us help you find your dream home! Who knows, maybe one day we’ll be laughing together on that rooftop mini-golf course! 
```


In [60]:

def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    #for chunk in stream in stream:
        #print(chunk.choices[0].delta.content or '',end ='')
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)


In [61]:
stream_brochure("Bogariagency", "http://bogariagency.com/")

Found links: {'links': [{'type': 'home page', 'url': 'http://bogariagency.com/'}, {'type': 'about page', 'url': 'http://bogariagency.com/about-farideh/'}, {'type': 'development page', 'url': 'http://bogariagency.com/development/'}, {'type': 'contact page', 'url': 'http://bogariagency.com/contact/'}, {'type': 'testimonials page', 'url': 'http://bogariagency.com/testimonials/'}, {'type': 'press page', 'url': 'http://bogariagency.com/press/'}]}


# Welcome to Bogari International Realty
*Where Luxury Meets Leisure in Real Estate Management... with a Dash of Fun!*

---

## About Us: The *Faye*-tastic Team
At **Bogari International Realty**, we don't just sell or manage properties; we craft unique experiences wrapped in luxury! Founded by the incredible **Farideh "Faye" Bogari**, our team has offices spanning Beverly Hills, New York, and the scenic streets of Vienna. Think of us as the *UN of Real Estate*—our mission? Making property dreams come true across borders!

---

## Our Properties: A Little Something for Everyone
From stunning homes that scream "Yes, please!" to sprawling commercial spaces that fit your every delight (we're talking rooftop mini-golf, people!), we've got it all:

- Residential estates so fine they’ll make your Pinterest boards blush.
- Commercial spaces that feel like they could double as a luxury hotel (because why not?).
- Development projects that we build from scratch—like a gourmet sandwich, but with walls!

---

## Customers: Who Do We Serve?
We cater to anyone with a taste for the finer things in life! Whether you’re a first-time homebuyer confused by the *MLS search* (Don’t worry, it's not the latest Major League Soccer league) or a seasoned investor looking to expand your portfolio, we’re here to help you every step of the way.

But don’t just take our word for it—here’s what some of our customers have to say:

> “Faye's commitment is unrivaled! She makes every deal feel like a walk in the park... if that park had luxurious villas!” - **James Jurdi**

> “Faye is seriously persistent, diligent, and she HAS A POSITIVE VIBE! I mean, who can negotiate an embassy deal with a smile?!” - **Hassan Shawaf** 

> “Living in the best home in California, complete with a rooftop golf course? Yes, please! Who said luxury can't be fun?” - **Josh Swartz**

---

## Company Culture: We're All About Faye-ly!
At Bogari, we like to keep it professional but fun—think *Corporate Champs meet the Office Funnies*. Each team member embodies what we like to call being "Faye-nomenal": 

- ***Collaborative***: We believe that team brainstorming sessions should include snacks. Lots of snacks.
- ***Client-focused***: Our clients are the stars of the show, and we roll out the red carpet (figuratively speaking... or literally, if it fits your property aesthetic!).
- ***Innovative***: We embrace new ideas faster than you can say "cookie dough ice cream!"

---

## Join the Faye-volution! Careers at Bogari
Have a knack for creating or managing luxurious spaces? Want to work in a place where laughter and innovation collide? Then we want you on our team!

Browse our website for the latest open positions, and get ready to say goodbye to mundane jobs. Here, we create luxury—one smile at a time!

---

## Contact Us
Ready to dive into your next property adventure? Call us at **+1.310.717.0029** or drop us a line at **info(at)Bogari(dotted)Agency**. Let’s make your dreams as big as your house can be!

---

*Bogari International Realty: The only real estate company where laughter and luxury go hand in hand!* 🏡✨